# Removing Dateless items

In [1]:
import pandas as pd
import datetime
import itertools

In [2]:
dates =pd.read_table("cit-HepPh-dates.txt", header = None)
dates.columns =['id', 'date']

In [3]:
raw = pd.read_table("Cit-HepPh.txt", header = None)

In [4]:
dataset =set(raw.iloc[:,1].tolist()+raw.iloc[:,0].tolist())

In [5]:
dateset =set(dates.iloc[:,0])

In [6]:
uniq = list(dataset & dateset)

In [7]:
isuniq = raw.isin(uniq)

In [8]:
data = raw[isuniq.iloc[:,1] & isuniq.iloc[:,0]]
data.columns =['id1', 'id2']

In [9]:
data.shape

(347268, 2)

In [10]:
raw.shape

(421578, 2)

In [11]:
data = data.merge(dates, left_on="id1", right_on = "id")

In [12]:
data = data.merge(dates, left_on="id2", right_on="id")

In [13]:
del data['id_x']
del data['id_y']

In [14]:
data['date_x'] = pd.to_datetime(data['date_x'])
data['date_y'] = pd.to_datetime(data['date_y'])

data  = data.iloc[data['date_x'].argsort(),:]

In [15]:
data = data[data.date_x>data.date_y]

In [16]:
data = data.reset_index()
del data['index']

In [34]:
data.shape

(347078, 4)

# Sample Creation

In [17]:
trainingsize= 10000
for i in range(data.shape[0]):
    if len(set(data['id1'].loc[0:i]))==trainingsize:
        print(i)
        train_end = i
        break
        
validationsize = 100
for i in range(data.shape[0]):
    if len(set(data['id1'].loc[train_end:i]))==validationsize:
        print(i)
        val_end = i
        break


testsize = 100
for i in range(data.shape[0]):
    if len(set(data['id1'].loc[val_end:i]))==validationsize:
        print(i)
        test_end = i
        break
        
        
#manual check, change  train end to 75052(-3)
#manual check, change val end to 76192(+48)
# change test end to  77395(+50)

75072
76079
77310


In [38]:
train_end = 75246
val_end = 76228
test_end = 77519

train_end_new = 75246
val_end_new = 76228
test_end_new = 77519


In [39]:
def f7(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

In [40]:
train = data.loc[0:train_end_new]

In [41]:
train.shape

(75247, 4)

In [42]:
len(f7(train.id1))

10003

In [43]:
len(f7(train.id2))

8653

In [56]:
a

{5: 4}

In [58]:
4 in a

False

In [50]:
train[train.id1 ==9205224].id2

0    9204213
Name: id2, dtype: int64

In [60]:
node_dict = {}
node = 1
for id1 in f7(train.id1):
    for id2 in train[train.id1 ==id1].id2:
        if id2 in node_dict:
            continue
        else:
            node_dict[id2] = node
            node +=1
    if id1 in node_dict:
        continue
    else:
        node_dict[id1] = node
        node +=1
            

In [77]:
inv_map = {v: k for k, v in node_dict.items()}

In [62]:
d = pd.Series(node_dict, name = "node")
d.index.name = "id"
d = d.reset_index()

In [64]:
train =train.merge(d, left_on ='id1', right_on = 'id')
train =train.merge(d, left_on ='id2', right_on = 'id')
del train['id_x']
del train['id_y']
train = train.sort_values(['date_x'])

In [75]:
train.to_csv('training.csv',index  = False)

In [82]:
train[train.id1 == 9610217] 

,id1,id2,date_x,date_y,node_x,node_y
69618,9610217,9411382,1996-10-02,1994-11-23,9238,3582


In [88]:
train[train.id2== 9610217]

,id1,id2,date_x,date_y,node_x,node_y
72393,9701327,9610217,1997-01-20,1996-10-02,10241,9238
72394,9706244,9610217,1997-06-04,1996-10-02,11430,9238
72395,9706401,9610217,1997-06-18,1996-10-02,11585,9238
72396,9706450,9610217,1997-06-23,1996-10-02,11633,9238


In [85]:
train.tail()


,id1,id2,date_x,date_y,node_x,node_y
74648,9706488,9701230,1997-06-24,1997-01-06,11676,10144
16835,9706484,9307291,1997-06-24,1993-07-18,11678,695
47453,9706496,9503484,1997-06-24,1995-03-31,11680,4548
57468,9706486,9511411,1997-06-24,1995-11-24,11666,6495
75246,9706487,9702314,1997-06-24,1997-02-12,11679,10453


In [80]:
for i in   [2351, 4459,  7051,  8251,  9215,  9238,  9485,  9890,  9965, 11089]:
    print(inv_map[i])

9405416
9503387
9602229
9606433
9609525
9610217
9610476
9608478
9612342
9704440


In [18]:
val = data.loc[train_end_new+1:val_end_new]

val_set =set(val['id1'].tolist()+val['id2'].tolist())
dictionary =dict(zip(val_set,range(1,len(val_set)+1)))
d = pd.Series(dictionary, name = "node")
d.index.name = "id"
d = d.reset_index()

val =val.merge(d,left_on ='id1', right_on = 'id')
val =val.merge(d, left_on ='id2', right_on = 'id')
del val['id_x']
del val['id_y']
val = val.sort_values(['date','id1'])

In [19]:
val.head()

,id1,id2,date,node_x,node_y
0,9706438,9408384,1997-06-20,466,981
136,9706431,9504215,1997-06-20,458,805
137,9706431,9602300,1997-06-20,458,699
138,9706431,9603351,1997-06-20,458,148
139,9706431,9303236,1997-06-20,458,659


In [20]:
val.tail()

,id1,id2,date,node_x,node_y
1066,9706522,9704335,1997-06-27,548,429
1065,9706522,9407347,1997-06-27,548,404
1064,9706522,9509225,1997-06-27,548,196
1075,9706530,9511387,1997-06-27,556,237
1139,9706527,9706235,1997-06-27,553,335


In [21]:
test = data.loc[val_end_new+1:test_end_new]

test_set =set(test['id1'].tolist()+test['id2'].tolist())
dictionary =dict(zip(test_set,range(1,len(test_set)+1)))
d = pd.Series(dictionary, name = "node")
d.index.name = "id"
d = d.reset_index()

test =test.merge(d,left_on ='id1', right_on = 'id')
test =test.merge(d, left_on ='id2', right_on = 'id')
del test['id_x']
del test['id_y']
test = test.sort_values(['date','id1'])

In [22]:
test.head()

,id1,id2,date,node_x,node_y
0,9706536,9303255,1997-06-28,521,634
34,9706534,9705290,1997-06-28,519,959
35,9706534,9705318,1997-06-28,519,979
36,9706534,9605367,1997-06-28,519,133
37,9706534,9607472,1997-06-28,519,167


In [23]:
test.tail()

,id1,id2,date,node_x,node_y
1042,9707273,9610431,1997-07-07,921,616
1043,9707273,9608441,1997-07-07,921,658
1044,9707273,9606423,1997-07-07,921,682
994,9707259,9705304,1997-07-07,908,969
1202,9707268,9612290,1997-07-07,917,493


In [24]:
train.to_csv('training.csv',index  = False)
val.to_csv('validation.csv',index  = False)
test.to_csv('test.csv',index  = False)